# AE1: Implementacja algorytmu genetycznego – opt. f-cji kwadratowej w R³
Adrianna Grudzień


Podstawowy algorytm genetyczny z mutacją gaussowską i krzyżowaniem jednopunktowym. Sprawdzić działanie algorytmu na funkcji x2+y2+2z2 oraz na pięciowymiarowej funkcji Rastrigina. 



Poczynione kroki:
1. Inicjalizacja populacji - N x-ów

In [36]:
import numpy as np

## Funkcje pomocnicze

In [28]:
def multiply_array(scalar, x):
    x_new = np.zeros(len(x))
    for i in range(len(x)):
        x_new[i] = x[i] + x[i]*scalar
    return x

## Funkcje

In [32]:
def fun1(x):
    'x - array with length=3'
    return np.square(x[0]) + np.square(x[1]) + 2*np.square(x[2])

def rastrigin_fun(x):
    'x - array'
    A = 10
    n = len(x)
    return A*n + np.sum(np.square(x) - multiply_array(A,np.cos(multiply_array(2*np.pi, x))))


In [54]:
class AE:
    def __init__(self, N, dim):
        self.N = N # liczność popoulacji
        self.dim = dim # wymiar obserwacji
        self._initialize_population()
        
    def _initialize_population(self):
        self.population = [np.random.uniform(0,1, size=(self.dim, self.N)) for i in range(self.N)]

In [53]:
ae = AE(50, 3)

50
